# RAG with Cohere CommandR model + citations

In [4]:
import os
import cohere
from dotenv import load_dotenv
load_dotenv()

co = cohere.Client(os.getenv('COHERE_API_KEY'))

In [5]:
import wikipedia

article = wikipedia.page('Dune Part Two')
text = article.content
print(f"The text has roughly {len(text.split())} words.")

The text has roughly 5797 words.


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create basic configurations to chunk the text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# Split the text into chunks with some overlap
chunks_ = text_splitter.create_documents([text])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

The text has been broken down in 101 chunks.


In [7]:
# Because the texts being embedded are the chunks we are searching over, we set the input type as search_doc
model="embed-english-v3.0"
response = co.embed(
    texts= chunks,
    model=model,
    input_type="search_document",
    embedding_types=['float']
)
embeddings = response.embeddings.float
print(f"We just computed {len(embeddings)} embeddings.")

We just computed 101 embeddings.


In [8]:
import numpy as np

vector_database = {i: np.array(embedding) for i, embedding in enumerate(embeddings)}
# { 0: array([...]), 1: array([...]), 2: array([...]), ..., 10: array([...]) }

In [9]:
query = "Name everyone involved in writing the script, directing, and producing 'Dune: Part Two'?"

In [13]:
# Because the text being embedded is the search query, we set the input type as search_query
response = co.embed(
    texts=[query],
    model=model,
    input_type="search_query",
    embedding_types=['float']
)
query_embedding = response.embeddings.float[0]
print("query_embedding: ", len(query_embedding))

query_embedding:  1024


In [17]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Calculate similarity between the user question & each chunk
similarities = [cosine_similarity(query_embedding, chunk) for chunk in embeddings]
print("similarity scores: ", similarities[:10])

# Get indices of the top 10 most similar chunks
sorted_indices = np.argsort(similarities)[::-1]

# Keep only the top 10 indices
top_indices = sorted_indices[:10]
print("Here are the indices of the top 10 chunks after retrieval: ", top_indices)

# Retrieve the top 10 most similar chunks
top_chunks_after_retrieval = [chunks[i] for i in top_indices]
print("Here are the top 10 chunks after retrieval: ")
for i, t in zip(top_indices, top_chunks_after_retrieval):
    print(f"\nIndex: {i}) Text: " + t[:100])

similarity scores:  [0.6888614728034463, 0.38807630598882875, 0.591677130062394, 0.6249606075622278, 0.3857665530943082, 0.24433414846393073, 0.38992649061156226, 0.32510833331345956, 0.30015660628306146, 0.42986330875799494]
Here are the indices of the top 10 chunks after retrieval:  [24 27 23  0 31  3 33 32 89  2]
Here are the top 10 chunks after retrieval: 

Index: 24) Text: stunt coordinator. Dune: Part Two was produced by Villeneuve, Mary Parent, and Cale Boyter, with Tan

Index: 27) Text: that."On October 26, 2021, Legendary officially greenlit Dune: Part Two, with a spokesperson for the

Index: 23) Text: series. Villeneuve ultimately secured a two-film deal with Warner Bros. Pictures, in the same style 

Index: 0) Text: Dune: Part Two is a 2024 American epic science fiction film directed and produced by Denis Villeneuv

Index: 31) Text: Eric Roth was hired to co-write the screenplay in April 2017 for the Dune films, and Jon Spaihts was

Index: 3) Text: theatrical and HBO Max rel

In [41]:
response = co.rerank(
    query=query,
    documents=top_chunks_after_retrieval,
    top_n=3,
    model="rerank-english-v2.0",
)

top_chunks_index_after_rerank = [result.index for result in response.results]
top_chunks_after_rerank = [top_chunks_after_retrieval[i] for i in top_chunks_index_after_rerank]
print("Here are the top 3 chunks after rerank: ")
for t in top_chunks_after_rerank:
    print("\n" + t)

Here are the top 3 chunks after rerank: 

Dune: Part Two is a 2024 American epic science fiction film directed and produced by Denis Villeneuve, who co-wrote the screenplay with Jon Spaihts. The sequel to Dune (2021), it is the second of a two-part adaptation of the 1965 novel Dune by Frank Herbert. It follows Paul Atreides as he unites with the Fremen people of the desert planet Arrakis to wage war against House Harkonnen. Timothée Chalamet, Zendaya, Rebecca Ferguson, Josh Brolin, Dave Bautista, Stellan Skarsgård, Charlotte Rampling, and Javier

stunt coordinator. Dune: Part Two was produced by Villeneuve, Mary Parent, and Cale Boyter, with Tanya Lapointe, Brian Herbert, Byron Merritt, Kim Herbert, Thomas Tull, Jon Spaihts, Richard P. Rubinstein, John Harrison, and Herbert W. Gain serving as executive producers and Kevin J. Anderson as creative consultant. Legendary CEO Joshua Grode confirmed in April 2019 that they plan to make a sequel, adding that "there's a logical place to stop t

In [42]:
# preamble containing instructions about the task and the desired style for the output.
preamble = """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

In [46]:
# retrieved documents
documents = [
    {"title": "chunk 0", "snippet": top_chunks_after_rerank[0]},
    {"title": "chunk 1", "snippet": top_chunks_after_rerank[1]},
    {"title": "chunk 2", "snippet": top_chunks_after_rerank[2]},
  ]

# get model response
response = co.chat(
  message=query,
  documents=documents,
  preamble=preamble,
  model="command-r",
  temperature=0.3
)

print("Final answer:")
print(response.text)

Final answer:
Here's a list of everyone involved in writing the script, directing, and producing 'Dune: Part Two':
- Denis Villeneuve (director and producer)
- Jon Spaihts (screenwriter and producer)
- Mary Parent (producer)
- Cale Boyter (producer)

The film also had a large crew including:
- Joe Walker (editor)
- Brad Riker (supervising art director)
- Patrice Vermette (production designer)
- Paul Lambert (visual effects supervisor)
- Gerd Nefzer (special effects supervisor)
- Thomas Struthers (stunt coordinator)
- Tanya Lapointe (executive producer)
- Brian Herbert (executive producer)
- Byron Merritt (executive producer)
- Kim Herbert (executive producer)
- Thomas Tull (executive producer)
- John Harrison (executive producer)
- Herbert W. Gain (executive producer)
- Richard P. Rubinstein (executive producer)
- Kevin J. Anderson (creative consultant)


In [47]:
print("Citations that support the final answer:")
for cite in response.citations:
  print(cite)

Citations that support the final answer:
start=103 end=108 text='Denis' document_ids=['doc_0']
start=109 end=119 text='Villeneuve' document_ids=['doc_0', 'doc_1']
start=120 end=129 text='(director' document_ids=['doc_0']
start=134 end=142 text='producer' document_ids=['doc_0', 'doc_1']
start=146 end=157 text='Jon Spaihts' document_ids=['doc_0']
start=158 end=171 text='(screenwriter' document_ids=['doc_0']
start=176 end=184 text='producer' document_ids=['doc_1']
start=188 end=199 text='Mary Parent' document_ids=['doc_1']
start=200 end=210 text='(producer)' document_ids=['doc_1']
start=213 end=224 text='Cale Boyter' document_ids=['doc_1']
start=225 end=235 text='(producer)' document_ids=['doc_1']
start=281 end=291 text='Joe Walker' document_ids=['doc_2']
start=292 end=300 text='(editor)' document_ids=['doc_2']
start=303 end=313 text='Brad Riker' document_ids=['doc_2']
start=314 end=340 text='(supervising art director)' document_ids=['doc_2']
start=343 end=359 text='Patrice Vermette' docu

In [50]:
def insert_citations_in_order(text, citations):
    """
    A helper function to pretty print citations.
    """
    offset = 0
    document_id_to_number = {}
    citation_number = 0
    modified_citations = []

    # Process citations, assigning numbers based on unique document_ids
    for citation in citations:
        citation_numbers = []
        for document_id in sorted(citation.document_ids):
            if document_id not in document_id_to_number:
                citation_number += 1  # Increment for a new document_id
                document_id_to_number[document_id] = citation_number
            citation_numbers.append(document_id_to_number[document_id])

        # Adjust start/end with offset
        start, end = citation.start + offset, citation.end + offset
        placeholder = ''.join([f'[{number}]' for number in citation_numbers])
        # Bold the cited text and append the placeholder
        modification = f'**{text[start:end]}**{placeholder}'
        # Replace the cited text with its bolded version + placeholder
        text = text[:start] + modification + text[end:]
        # Update the offset for subsequent replacements
        offset += len(modification) - (end - start)

    # Prepare citations for listing at the bottom, ensuring unique document_ids are listed once
    unique_citations = {number: doc_id for doc_id, number in document_id_to_number.items()}
    citation_list = '\n'.join([f'[{doc_id}] source: "{documents[doc_id - 1]["snippet"]}"' for doc_id, number in sorted(unique_citations.items(), key=lambda item: item[1])])
    text_with_citations = f'{text}\n\n{citation_list}'

    return text_with_citations


print(insert_citations_in_order(response.text, response.citations))

Here's a list of everyone involved in writing the script, directing, and producing 'Dune: Part Two':
- **Denis**[1] **Villeneuve**[1][2] **(director**[1] and **producer**[1][2])
- **Jon Spaihts**[1] **(screenwriter**[1] and **producer**[2])
- **Mary Parent**[2] **(producer)**[2]
- **Cale Boyter**[2] **(producer)**[2]

The film also had a large crew including:
- **Joe Walker**[3] **(editor)**[3]
- **Brad Riker**[3] **(supervising art director)**[3]
- **Patrice Vermette**[3] **(production designer)**[3]
- **Paul Lambert**[3] **(visual effects supervisor)**[3]
- **Gerd Nefzer**[3] **(special effects supervisor)**[3]
- **Thomas Struthers**[3] **(stunt coordinator)**[3]
- **Tanya Lapointe**[2] **(executive producer)**[2]
- **Brian Herbert**[2] **(executive producer)**[2]
- **Byron Merritt**[2] **(executive producer)**[2]
- **Kim Herbert**[2] **(executive producer)**[2]
- **Thomas Tull**[2] **(executive producer)**[2]
- **John Harrison**[2] **(executive producer)**[2]
- **Herbert W. Gain**[2

Reference: https://github.com/cohere-ai/notebooks/blob/main/notebooks/Vanilla_RAG.ipynb